Sentence similarity
Dataset: STS

In [1]:
import torch
from datasets import load_dataset, Dataset
import nltk
import re
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses

from nltk.corpus import words
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
nltk.download('words', quiet=True)
nltk.download('wordnet', quiet=True)
# vocab = set(words.words())
from nltk.stem import *

from preprocessing import clean_sentence, vocab, clean_dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from transformers.integrations import TensorBoardCallback

In [3]:
ds = load_dataset("tabilab/biosses")

In [4]:
train_test = ds['train'].train_test_split(test_size=0.2, seed=42)
train_validate = train_test['train'].train_test_split(test_size=0.2, seed=42)
train_set = train_validate['train']
validate_set = train_validate['test']
test_set = train_test['test']

In [5]:
stemmer = PorterStemmer()

In [6]:
stemmed_words = set(map(lambda word: stemmer.stem(word), words.words()))

In [7]:
cleaned_train, unknown_words = clean_dataset(train_set, stemmed_words)
cleaned_validation, _ = clean_dataset(validate_set, stemmed_words)
cleaned_test, _ = clean_dataset(test_set, stemmed_words)

In [8]:
print(unknown_words)
print(len(unknown_words))

{'Oct4', 'p19(p14)/ARF', 'cycle-dependent', 'qRT-PCR', 'RIP1', 'upregulated', '(BRG-associated', 'miR-126', 'oncogene-driven', 'Eukaryotic', '(CD44', 'H-RASV12', 'Up-regulation', 'miR-21)', 'epithelial-mesenchymal', 'hmC', 'AML', 'miR-143', '(', 'Cyclin', 'Affymetrix', '(together', 'miR-133b', 'E2F1', 'erythropoietin', 'miR-146b', 'miR-204', 'prometastatic', 'miR-145', 'microtubule-nucleating', '[18]', 'miRNA-regulated', 'et', 'SWI/SNF-regulated', '“cyclin', 'EGFR', 'EC', 'TBK1', 'ribonucleotide', 'StemBase', 'pCAG-GFP', 'Renilla-3′', 'proteasome', 'non-neural', 'dose-dependent', 'Skbr3', 'germ-cell', 'RIPK3', 'respectively)', 'GTPase', 'B-ALL', 'TargetScan', 'miR-155', 'SDS-PAGE', 'granulopoiesis', 'factor)', 'actin-related', 'HEK293T', 'large-scale', '(PTC)', 'Neuro-2a', 'LATS2-depletion', 'APC-dependent', 'ligase', 'ubiquitin', 'T47D', 'ductal', 'miR-204-miRVec', '(RIP', 'up-regulated', 'ESCs', 'RIP3', '72', 'caspase', 'MOE430A', 'TRAF6', 'I/A', 'dysregulation', 'tumorigenic', 'BAF5

In [9]:
torch.cuda.empty_cache()

In [10]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainingArguments
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction
from sentence_transformers.losses import CoSENTLoss

# Load a model to train/finetune
model = SentenceTransformer('all-MiniLM-L6-v2', device="cuda")

# Initialize the CoSENTLoss
# This loss requires pairs of text and a float similarity score as a label
loss = CoSENTLoss(model)

In [11]:
def evaluate_model(model_, test_set_, name="BIOSSES_test"):
    test_evaluator = EmbeddingSimilarityEvaluator(
        sentences1=test_set_["sentence1"],
        sentences2=test_set_["sentence2"],
        scores=test_set_["score"],
        name=name,
    )
    return test_evaluator(model)

In [12]:
train_s1 = cleaned_train['sentence1']
train_s2 = cleaned_train['sentence2']
validation_s1 = cleaned_validation['sentence1']
validation_s2 = cleaned_validation['sentence2']

In [13]:
# train_emb1 = model.encode(train_s1, normalize_embeddings=True)
# train_emb2 = model.encode(train_s2, normalize_embeddings=True)
# validation_emb1 = model.encode(validation_s1, normalize_embeddings=True)
# validation_emb2 = model.encode(validation_s2, normalize_embeddings=True)

In [14]:
# validation_similarities = model.similarity_pairwise(validation_emb1, validation_emb2)
# print(validation_similarities)
ground_truth_validation = cleaned_validation["score"]
# print(ground_truth_validation)
# print(loss.forward(
#     {"sentence1": cleaned_validation["sentence1"],
#      "sentence2": cleaned_validation["sentence2"]}, 
#     ground_truth_validation))

In [15]:
test_similarities = model.similarity_pairwise(
    model.encode(cleaned_test['sentence1'], normalize_embeddings=True),
      model.encode(cleaned_test['sentence2'], normalize_embeddings=True))
print(test_similarities)
ground_truth_test = cleaned_test["score"]
print(ground_truth_test)

tensor([0.6227, 0.7996, 0.4542, 0.7373, 0.5717, 0.9421, 0.4272, 0.3671, 0.2034,
        0.7460, 0.5736, 0.5206, 0.8329, 0.6441, 0.5521, 0.7422, 0.3377, 0.7593,
        0.4291, 0.3967])
[0.20000004768371582, 0.7999999523162842, -0.30000001192092896, 0.6000000238418579, 0.5, 1.0, 0.0, -0.30000001192092896, -0.10000002384185791, 0.7000000476837158, 0.0, 0.0, 0.8999999761581421, 0.7000000476837158, 0.7999999523162842, 0.5, 0.20000004768371582, 0.6000000238418579, 0.20000004768371582, -0.30000001192092896]


C:\Users\pc\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [16]:
import datetime

In [17]:
num_train_epochs=1000
per_device_train_batch_size=2
learning_rate=1e-6

In [18]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/all-MiniLM-L6-v2/"+str(datetime.datetime.now()).split()[1].split('.')[0].replace(':', '-'),
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=100,
    learning_rate=learning_rate,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    # batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=0.05,
    save_strategy="steps",
    save_steps=0.05,
    # save_total_limit=3,
    load_best_model_at_end=True,
    logging_steps=0.05,
    run_name='{num_train_epochs}_{per_device_train_batch_size}_{learning_rate}',  # Will be used in W&B if `wandb` is installed
)

In [19]:
dev_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=cleaned_validation["sentence1"],
    sentences2=cleaned_validation["sentence2"],
    scores=cleaned_validation["score"],
    name="BIOSSES_validate",
)
test_evaluator = EmbeddingSimilarityEvaluator(
    sentences1=cleaned_test["sentence1"],
    sentences2=cleaned_test["sentence2"],
    scores=cleaned_test["score"],
    name="BIOSSES_test",
)

In [20]:
print(evaluate_model(model, cleaned_test)['BIOSSES_test_pearson_cosine'])
print(evaluate_model(model, cleaned_train)['BIOSSES_test_pearson_cosine'])
print(evaluate_model(model, cleaned_validation)['BIOSSES_test_pearson_cosine'])

0.8255906849948111
0.8178701842922809
0.8034244246101594


In [21]:
tensor_board_callback = TensorBoardCallback()

In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=cleaned_train,
    eval_dataset=cleaned_validation,
    loss=loss,
    args=args,
    evaluator=dev_evaluator,
    callbacks=[tensor_board_callback]
)
trainer.train()

In [26]:
test_results = test_evaluator(model)
test_results

{'BIOSSES_test_pearson_cosine': 0.8331904985026963,
 'BIOSSES_test_spearman_cosine': 0.802586556268141}

In [24]:
model.save_pretrained("models/all-MiniLM-L6-v2/final/{num_train_epochs}-{per_device_train_batch_size}-{learning_rate}".format(
    learning_rate=learning_rate, num_train_epochs=num_train_epochs, per_device_train_batch_size=per_device_train_batch_size
))

In [27]:
print(evaluate_model(model, cleaned_test)['BIOSSES_test_pearson_cosine'])
print(evaluate_model(model, cleaned_train)['BIOSSES_test_pearson_cosine'])
print(evaluate_model(model, cleaned_validation)['BIOSSES_test_pearson_cosine'])

0.8331904985026963
0.9704619857775055
0.8472240995034808
